In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import streamlit as st
import plotly.express as px
#import plotly.express == 5.5.0

In [ ]:
df = pd.read_csv('pompeia.csv', sep = ';')

In [ ]:
#Dividir a coluna do ultimo status em duas novas colunas no dataframe original, exluindo a coluna original
df[['data_ultimo_status', 'hora_ultimo_status']] = df["Último Status"].str.split(' ', expand = True)
df.drop(columns =["Último Status"], inplace = True) 

In [ ]:
# Convertendo strings para date e time
df['Nascimento'] = pd.to_datetime(df['Nascimento'], format="%d/%m/%Y")
df['data_ultimo_status'] = pd.to_datetime(df['data_ultimo_status'], format="%d/%m/%Y")
df['hora_ultimo_status'] = pd.to_datetime(df['hora_ultimo_status'], format="%H:%M")

In [ ]:
#O cliente sedentário é o primeiro registro na ordenação por data do último status
cliente_sedentario = df[['Código','data_ultimo_status']].sort_values(by=['data_ultimo_status'])
cliente_sedentario = cliente_sedentario.head(1)

In [ ]:
#1. Qual é o cliente que não vai a mais tempo?
cliente_sedentario

In [ ]:
consultores = df[['Consultor']].groupby('Consultor').agg(Total=('Consultor','count')).sort_values(by='Total', ascending=False)
consultores.head(7)

In [ ]:
#2. O consultor que tem mais clientes
consultor = consultores.head(1)
consultor

In [ ]:
professores = df[['Professor']].groupby('Professor').agg(Total=('Professor','count')).sort_values(by='Total', ascending=False)
professores.head(8)

In [ ]:
#3. Professor que tem mais alunos
professor = professores.head(2)
professor

In [ ]:
#Corrigir 1 registro específico no DF
correcao = {'FUNCIONAL + MUSCULAÇÃO, NATAÇÃO':'FUNCIONAL + MUSCULAÇÃO + NATAÇÃO'}
df.replace(correcao, inplace= True)
#Cria um DF limpo
modalidades = pd.DataFrame()
#Separa os registros em colunas diferentes
modalidades[['sexo']] = df['Sexo']
modalidades[['Modalidade1','Modalidade2','Modalidade3']] = df['Modalidade'].str.split('+', expand = True)
#Padronização dos registros no dataframe modalidade
correcao = {'MUSCULAÇÃO ': 'MUSCULAÇÃO',' MUSCULAÇÃO':'MUSCULAÇÃO',' MUSCULAÇÃO ':'MUSCULAÇÃO','FUNCIONAL ':'FUNCIONAL',' FUNCIONAL':'FUNCIONAL','NATAÇÃO ':'NATAÇÃO',' NATAÇÃO':'NATAÇÃO','-':'PLANO COMPLETO'}
modalidades.replace(correcao, inplace= True)

In [ ]:
#4. Qual é a modalidade mais consumida
modalidade = pd.DataFrame(modalidades['Modalidade1'].value_counts())
modalidade